In [1]:
import os

import numpy as np
import xarray as xr

In this notebook we create a dataset containing the zonal mean fields of quantities relevant to the residual circulation &mdash; namely, $u$, $v$, $w$, and $T$ &mdash; as well as the residual circulation components $\bar{v}^\ast$ and $\bar{w}^\ast$ themselves. Seviour et al. (2012) suggest that six-hourly resolution is necessary to capture the diurnal variability of the residual upwelling. Fortunately, there is an ERA5 dataset on disk ([633.1](https://rda.ucar.edu/datasets/ds633.1/)) of monthly mean data derived from the original six-hourly resolution.

We begin by setting the appropriate directory and enumerating the available years.

In [2]:
era_dir = '/gpfs/fs1/collections/rda/data/ds633.1/e5.moda.an.pl'
years = [int(x) for x in sorted(os.listdir(era_dir))]

print(f'Found {len(years)} years of ERA5 data, starting with {years[0]} and ending with {years[-1]}.')

Found 43 years of ERA5 data, starting with 1979 and ending with 2021.


Now, we will loop over the directories for each year in the dataset and pull the names of the files corresponding to the variables we want.

In [3]:
names = ['u', 'v', 'w', 'T', 'Z']

ds_fnames = []
for year in years:
    year_dir = f'{era_dir}/{year}'
    fnames = [x for x in os.listdir(year_dir) if x.endswith('nc')]
    
    for name in names:
        fname = [x for x in fnames if f'_{name.lower()}.' in x][0]
        ds_fnames.append(f'{year_dir}/{fname}')
        
print(f'Found {len(ds_fnames)} files out of an expected {len(years) * len(names)}.')

Found 215 files out of an expected 215.


Now, `xr.open_mfdataset` gives us an easy way to create an `xr.Dataset` containing the data from all the files above without loading everything into memory at once, which would be prohibitive. Below we open such a dataset and prepare it for use in calculating the residual circulation. Seviour et al. (2012) use a log-pressure vertical coordinate
$$z \equiv -H \log \frac{p}{p_{\textrm{surf}}}$$
with scale height $H = 6800 \textrm{ m}$. (At least, I believe that's what they mean &mdash; their description of the vertical coordinate seems to contain some typos.) They also define the reference density profile
$$\rho = \exp \left(-\frac{z}{H}\right)$$
We convert the dataset's vertical coordinate to $z$ defined above, since most the vertical differentiation we perform later will be with respect to $z$. We also convert the latitude coordinate to be in radians, so that trigonometric operations on the latitude are easier later on.

In [4]:
with xr.open_mfdataset(ds_fnames, combine='by_coords') as ds:
    ds = ds.rename({
        'level' : 'z',
        'U' : 'u',
        'V' : 'v',
        'W' : 'w',
        'Z' : 'Phi'
    }).drop('utc_date')
    
    p = (100 * ds['z']).assign_attrs(units='Pa')
    lat = (np.pi * ds['latitude'] / 180).assign_attrs(units='radians_north')
    
    H, p_surf = 6800, p[-1]
    z = (-H * np.log(p / p_surf)).assign_attrs(units='meters')
    rho = np.exp(-z / H)
    
    ds['p'], ds['rho'] = p, rho
    ds = ds.assign_coords(z=z, latitude=lat)
    
    p, lat, rho = ds['p'], ds['latitude'], ds['rho']
    u, v, w_cartesian = ds['u'], ds['v'], ds['w']
    T, h = ds['T'], ds['Phi'] / 9.8
    
    display(ds)

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 516, z: 37)
Coordinates:
  * latitude   (latitude) float64 1.571 1.566 1.562 ... -1.562 -1.566 -1.571
  * z          (z) float64 4.697e+04 4.226e+04 3.95e+04 ... 348.8 172.2 -0.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
Data variables:
    T          (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 721, 1440), chunksize=(12, 37, 721, 1440)>
    u          (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 721, 1440), chunksize=(12, 37, 721, 1440)>
    v          (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 721, 1440), chunksize=(12, 37, 721, 1440)>
    w          (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 721, 1440), chunksize=(12, 37, 721, 1440)>
    Phi        (time, z, latitude, longitude) float32 dask.array<shape=(516, 37, 72

However, the ERA5 vertical velocity is with respect to the Cartesian vertical coordinate, which I'll denote by $h$. By the chain rule we of course have
$$\frac{\mathrm{d} z}{\mathrm{d} t} = \frac{\mathrm{d} z}{\mathrm{d} h}\frac{\mathrm{d} h}{\mathrm{d} t}$$
where the left-hand side is the vertical velocity we want to analyze and the last factor on the right-hand side is the ERA5 vertical velocity. In the cell below, we compute the desired $w$ field.

In [5]:
dzdh = 1 / h.differentiate('z')
w = dzdh * w_cartesian

Next, we compute the potential temperature
$$\theta = \left(\frac{p_{\textrm{surf}}}{p}\right)^{R / c_{\mathrm{p}}}T$$
as well as the buoyancy frequency $N^2 = \bar{\theta}_z$. Note that we take $R / c_{\mathrm{p}} = 0.286$.

In [6]:
theta = T * ((p_surf / p) ** 0.286)
theta_bar = theta.mean('longitude')
N2 = theta_bar.differentiate('z')

We are now in a position to compute the components of the residual circulation according to
$$\bar{v}^\ast = \bar{v} - \frac{1}{\rho}\left(\frac{\rho\overline{v'\theta'}}{N^2}\right)_z$$
and
$$\bar{w}^\ast = \bar{w} + \frac{1}{a\cos\vartheta}\left(\cos\vartheta \frac{\overline{v'\theta'}}{N^2}\right)_{\vartheta}$$
where $\vartheta$ is the latitude and $a$ is the radius of the Earth.

In [7]:
v_bar = v.mean('longitude')
w_bar = w.mean('longitude')

v_prime = v - v_bar
w_prime = w - w_bar

theta_prime = theta - theta_bar
heat_flux = (v_prime * theta_prime).mean('longitude')

a, cos_lat = 6.37e6, np.cos(lat)
v_star = v_bar - (rho * heat_flux / N2).differentiate('z') / rho
w_star = w_bar + (cos_lat * heat_flux / N2).differentiate('latitude') / (a * cos_lat)

Before saving to disk, we'll also compute the Eliassen-Palm flux divergence, scaled as in Seviour et al. (2012) like
$$\textrm{EPFD} = \frac{\nabla \cdot \mathbf{F}}{\rho a \cos\vartheta}$$
where the horizontal and vertical components of the Eliassen-Palm flux are
$$F^{(\vartheta)} = \rho a \cos\vartheta \left(\bar{u}_z \frac{\overline{v'\theta'}}{N^2} - \overline{u'v'}\right)$$
and
$$F^{(z)} = \rho a \cos\vartheta \left[ \left(f - \frac{(\bar{u} \cos\vartheta)_\vartheta}{a\cos\vartheta}\right) \frac{\overline{v'\theta'}}{N^2} - \overline{u'w'} \right]$$

In [8]:
u_bar = u.mean('longitude')
u_prime = u - u_bar

uv_bar = (u_prime * v_prime).mean('longitude')
uw_bar = (u_prime * w_prime).mean('longitude')

Omega = 7.292e-5
f = 2 * Omega * np.sin(lat)
offset = (u_bar * cos_lat).differentiate('latitude') / (a * cos_lat)

F_lat = (u_bar.differentiate('z') * heat_flux / N2 - uv_bar) * rho * a * cos_lat
F_z = ((f - offset) * heat_flux / N2 - uw_bar) * rho * a * cos_lat
EPFD = (F_lat.differentiate('latitude') + F_z.differentiate('z')) / (rho * a * cos_lat)

Now, we'll assemble a dataset with the circulation features of note and save it to disk after taking a zonal mean to keep the size manageable. Note that it is only at this point that the actual calculations are done, so the `to_netcdf` call below takes some time. Before saving, we also go back to a pressure vertical coordinate and latitude expressed in degrees, for ease of analysis later.

In [9]:
ds = xr.Dataset({
    'u' : u,
    'v' : v,
    'w' : w_cartesian,
    'T' : T,
    'h' : h,
    'theta' : theta,
    'N2' : N2,
    'heat_flux' : heat_flux,
    'v_star' : v_star,
    'w_star' : w_star,
    'EPFD' : EPFD
}).mean('longitude').rename({'z' : 'pressure'})

p = (p_surf * np.exp(-ds['pressure'] / H) / 100).assign_attrs(units='hPa')
lat = (180 * ds['latitude'] / np.pi).assign_attrs(units='degrees_north')
ds = ds.assign_coords(pressure=p, latitude=lat).drop('z')

display(ds)
ds.to_netcdf('/glade/work/dconnell/brewson/data.nc')

<xarray.Dataset>
Dimensions:    (latitude: 721, pressure: 37, time: 516)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * pressure   (pressure) float64 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
Data variables:
    u          (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    v          (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    w          (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    T          (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    h          (time, pressure, latitude) float32 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    theta      (time, pressure, latitude) float64 dask.array<shape=(516, 37, 721), chunksize=(12, 37, 721)>
    N2         (tim